In [1]:
import pandas as pd

/home/joel/anaconda3/envs/mds/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
# ¿Es activa?
# tine un puente salino y tiene como inhibidor al ATP
import MDAnalysis
ref_id = "1fin"
dfg_out = "5a14"
pdbs_ENS_dir = "../ARCHIVOS/CRISTALES/PROT_CDK2_PREPARADAS/PREP_ALL_ENSAMBLE/"
ref_structure_file = pdbs_ENS_dir + ref_id + "_ENS.pdb"
ref_dfgOut_file = pdbs_ENS_dir + dfg_out + "_ENS.pdb"

In [2]:
import MDAnalysis
from MDAnalysis.analysis.dihedrals import Dihedral
import MDAnalysis.analysis.hbonds
import numpy as np

lista_res = {}

pdb_id_list = ["4gcj", "1aq1", "4fkw", "3pxr", "1fin", "5a14"]
pdbs_ENS_dir = "../ARCHIVOS/CRISTALES/PROT_CDK2_PREPARADAS/PREP_ALL_ENSAMBLE/"

for pdb_id in pdb_id_list:
    pdb_id_file = pdbs_ENS_dir + pdb_id + "_ENS.pdb"
    u = MDAnalysis.Universe(pdb_id_file)

    # ¿Es activa? o dfgout, será activa si tiene un puente salino entre Lys33 y Glu51 y no es DFGOUT
    hb_K33_Q51 = MDAnalysis.analysis.hbonds.HydrogenBondAnalysis(u, 
                            'resnum 33', 'resnum 51', distance=3.0, angle=120.0)
    hb_K33_Q51.run()
    hb_K33_Q51.generate_table()
    if len(hb_K33_Q51.table) > 0:
        # Es DFG-out
        phi_res_D145 = [ u.residues[144].phi_selection() ]
        phi_value = ( Dihedral(phi_res_D145).run() ).angles[0][0]
        if phi_value < 0:
            print(pdb_id, "Es dfg-out")
            d = {pdb_id: "dfg_out"}
        else:
            print(pdb_id, "Es activa")
            d = {pdb_id: "active"}
    # Entonces es abierta, inhibidora A o B
    else:
        aC = u.select_atoms("resnum 46-55").center_of_geometry()
        B4 = u.select_atoms("resnum 76-81").center_of_geometry()
        dist_aC_B4 = np.linalg.norm(B4 - aC)
        # ¿Es abierta?
        if dist_aC_B4 > 14.5:
            print(pdb_id, "Es abierta")
            d = {pdb_id: "open"}
        else:
            hb_helix_ALoop = MDAnalysis.analysis.hbonds.HydrogenBondAnalysis(u, 
                             'resnum 147-155', 'resnum 147-155', distance=3.0, angle=120.0)
            hb_helix_ALoop.run()
            hb_helix_ALoop.generate_table()
             # Entonces es inicativa A?
            if len( hb_helix_ALoop.table ) > 1:
                print(pdb_id, "Es Inactiva A")
                d = {pdb_id: "inactive_A"}
            else:
                print(pdb_id, "Es Inactiva B")
                d = {pdb_id: "inactive_B"}
    lista_res.update(d)

pd.Series(lista_res)

/home/joel/anaconda3/envs/mds/lib/python3.6/site-packages/MDAnalysis/analysis/hbonds/hbond_analysis.py:649: SelectionWarning: No acceptors found in selection 2. You might have to specify a custom 'acceptors' keyword. Selection will update so continuing with fingers crossed.
  warnings.warn(errmsg, category=SelectionWarning)
/home/joel/anaconda3/envs/mds/lib/python3.6/site-packages/MDAnalysis/analysis/hbonds/hbond_analysis.py:649: SelectionWarning: No donors found in selection 2. You might have to specify a custom 'donors' keyword. Selection will update so continuing with fingers crossed.
  warnings.warn(errmsg, category=SelectionWarning)
/home/joel/anaconda3/envs/mds/lib/python3.6/site-packages/MDAnalysis/coordinates/base.py:836: UserWarning: Reader has no dt information, set to 1.0 ps
  warnings.warn("Reader has no dt information, set to 1.0 ps")


4gcj Es Inactiva B
1aq1 Es Inactiva B
4fkw Es Inactiva B
3pxr Es Inactiva A
1fin Es activa
5a14 Es dfg-out


NameError: name 'pd' is not defined

In [13]:
u = MDAnalysis.Universe(ref_structure_file)
g = MDAnalysis.Universe(ref_dfgOut_file)
ags_1 = [res.phi_selection() for res in u.residues[144:145]]
ags_2 = [res.phi_selection() for res in g.residues[144:145]]

from MDAnalysis.analysis.dihedrals import Dihedral
r = Dihedral(ags_1).run()
s = Dihedral(ags_2).run()
print(r.angles, s.angles)

u.residues[144].phi_selection()
ags_1

[[60.17351814]] [[55.29519936]]


[<AtomGroup with 4 atoms>]

In [27]:
ags_1 = [res.phi_selection() for res in u.residues[144:145]]
dir(a)
print(Dihedral(ags_1).run())

In [35]:
u.universe.

<Universe with 4848 atoms>

In [21]:
import pytraj as pt
#traj = pt.load(ref_structure_file)
pt.calc_center_of_geometry(traj, ":46-55").shape
#type(traj)

(402, 3)

In [4]:
#traj = pt.load(ref_dfgOut_file)
pt.calc_phi(traj, resrange='145')

<pytraj.DatasetList with 1 datasets>
phi:145
[60.17326784]

In [29]:
import numpy as np
a = np.random.rand(90, 3) * 100
b = np.random.rand(90, 3) * -100

np.linalg.norm(a - b)

1754.9371967158077

In [79]:
file = '/home/joel/Documentos/Doctorado/DOCTO_TESIS/Proteinas_Modelo/CDK2/CDK2/ARCHIVOS/CRISTALES/VOLUMEN/CDK2_VOL_PISANI_402/PDB_402_cdk2_PISANI_ALL.pdb'

traj = pt.load(file)
print(traj.n_frames)
a = pt.calc_phi(traj, resrange='145')

a.values

402


array([50.9327455 , 60.64897315, 73.13150104, ..., 84.12745719,
       81.08172202, 71.25038634])

In [17]:
traj.n_frames

402

In [18]:
a = pt.search_hbonds(traj, ':33,51', distance=5.0, angle=145.0)
a.to_dict

<bound method BaseDataHolder.to_dict of <pytraj.hbonds.DatasetHBond
donor_acceptor pairs : 6>>

In [31]:
pt.distance(traj, ':33@NZ :51@CD')

array([13.82386476, 15.51196396, 15.90498947, ..., 18.35620105,
       17.79975969, 17.97402047])

In [68]:
sec_stc = pt.dssp_analysis.dssp(traj,  ':146-156')
sec_stc[0][0]
len(sec_stc[1][:])


402

In [80]:
print(sec_stc[1][1])
    
z = np.char.count(sec_stc[1], 'H').sum(axis=1) >= 4
y = a.values < 0
y

['0' '0' 'H' ... '0' '0' '0']


array([False, False, False, ..., False, False, False])

In [12]:
# PAra comprobar si es Inactivo A o B
import MDAnalysis
import MDAnalysis.analysis.hbonds

h = MDAnalysis.analysis.hbonds.HydrogenBondAnalysis(u, 'resnum 147-155', 'resnum 147-155', distance=4.0, angle=120.0)
print(h.run())
print(h.generate_table())
print(h)
import pandas as pd
len(h.table)

None
None


1

In [95]:
# PAra comprobar si es Inactivo A o B
import MDAnalysis
import MDAnalysis.analysis.hbonds

h = MDAnalysis.analysis.hbonds.HydrogenBondAnalysis(u, 'resnum 33', 'resnum 51', distance=3.0, angle=120.0)
h.run()
h.generate_table()
len(h.table)

1

In [116]:
# tine un puente salino y tiene como inhibidor al ATP
import MDAnalysis
ref_id = "1aq1"
dfg_out = "1aq1"
pdbs_ENS_dir = "../ARCHIVOS/CRISTALES/PROT_CDK2_PREPARADAS/PREP_ALL_ENSAMBLE/"
ref_structure_file = pdbs_ENS_dir + ref_id + "_ENS.pdb"
ref_dfgOut_file = pdbs_ENS_dir + dfg_out + "_ENS.pdb"

u = MDAnalysis.Universe(ref_structure_file)
g = MDAnalysis.Universe(ref_dfgOut_file)

# Para comprobar si es abierto
import numpy as np
aC = g.select_atoms("resnum 46-55").center_of_geometry()
b4 = g.select_atoms("resnum 76-81").center_of_geometry()

print(np.linalg.norm(b4 - aC) > 14)
#print(aC, b4)

False
